In [ ]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *
from pruning.structured import *

In [2]:
dataset = CIFAR10()
train_loader, test_loader = dataset.get_dataloader()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#for UnstructuredL1normPrune

model = AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = UnstructuredL1normPrune(model, 5, train_loader, criterion, optimizer, 0.5)

#train and prune the model and return both the original and pruned model
original_model, pruned_model = trainer.train_prune_retrain()


 20%|██        | 1/5 [00:47<03:09, 47.35s/it]

Epoch: 1 | train_loss: 6.4776 | 


 40%|████      | 2/5 [01:19<01:55, 38.53s/it]

Epoch: 2 | train_loss: 6.1819 | 


 60%|██████    | 3/5 [01:51<01:11, 35.63s/it]

Epoch: 3 | train_loss: 6.0183 | 


 80%|████████  | 4/5 [02:23<00:34, 34.24s/it]

Epoch: 4 | train_loss: 5.8902 | 


100%|██████████| 5/5 [02:56<00:00, 35.23s/it]

Epoch: 5 | train_loss: 5.7717 | 
Training is done
Pruning is done


In [4]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ",accuracy(pruned_model, test_loader))
print("Number of non zero weights in original model : ",non_zero_weights(original_model))
print("Number of non zero weights in pruned model : ",non_zero_weights(pruned_model))
print("Compression Ratio : ",compression_ratio(original_model, pruned_model))

Accuracy of original model :  [0.6778]
Accuracy of pruned model :  [0.5124]
Number of non zero weights in original model :  23262912
Number of non zero weights in pruned model :  11631456
Compression Ratio :  0.5


In [6]:
#for Decay pruning with reverse=False
model =AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = DecayPrune(model, 5, train_loader, criterion, optimizer, 0.8,0.05,False) #init pruning rate = 0.5, decay = 0.1

#train and prune the model and return both the original and pruned model
original_model, pruned_model = trainer.train_prune_retrain()

 20%|██        | 1/5 [00:32<02:09, 32.40s/it]

Epoch: 1 | train_loss: 6.4923 | 


 40%|████      | 2/5 [01:05<01:37, 32.55s/it]

Epoch: 2 | train_loss: 6.2048 | 


 60%|██████    | 3/5 [01:37<01:05, 32.59s/it]

Epoch: 3 | train_loss: 6.0384 | 


 80%|████████  | 4/5 [02:10<00:32, 32.53s/it]

Epoch: 4 | train_loss: 5.9064 | 


100%|██████████| 5/5 [02:42<00:00, 32.52s/it]

Epoch: 5 | train_loss: 5.7742 | 
Training is done
Pruning is done


In [7]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ",accuracy(pruned_model, test_loader))
print("Number of non zero weights in original model : ",non_zero_weights(original_model))
print("Number of non zero weights in pruned model : ",non_zero_weights(pruned_model))
print("Compression Ratio : ",compression_ratio(original_model, pruned_model))

Accuracy of original model :  [0.691]
Accuracy of pruned model :  [0.2405]
Number of non zero weights in original model :  23262912
Number of non zero weights in pruned model :  17489857
Compression Ratio :  0.7518343791181431


In [8]:
#for Decay pruning with reverse=True
model =AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = DecayPrune(model, 5, train_loader, criterion, optimizer, 0.8,0.05,True) #init pruning rate = 0.5, decay = 0.1

#train and prune the model and return both the original and pruned model
original_model, pruned_model = trainer.train_prune_retrain()

 20%|██        | 1/5 [00:32<02:10, 32.69s/it]

Epoch: 1 | train_loss: 6.4777 | 


 40%|████      | 2/5 [01:05<01:38, 32.67s/it]

Epoch: 2 | train_loss: 6.1907 | 


 60%|██████    | 3/5 [01:38<01:05, 32.76s/it]

Epoch: 3 | train_loss: 6.0286 | 


 80%|████████  | 4/5 [02:10<00:32, 32.66s/it]

Epoch: 4 | train_loss: 5.8960 | 


100%|██████████| 5/5 [02:43<00:00, 32.63s/it]

Epoch: 5 | train_loss: 5.7756 | 
Training is done
Pruning is done


In [9]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ",accuracy(pruned_model, test_loader))
print("Number of non zero weights in original model : ",non_zero_weights(original_model))
print("Number of non zero weights in pruned model : ",non_zero_weights(pruned_model))
print("Compression Ratio : ",compression_ratio(original_model, pruned_model))

Accuracy of original model :  [0.6752]
Accuracy of pruned model :  [0.2034]
Number of non zero weights in original model :  23262912
Number of non zero weights in pruned model :  17489857
Compression Ratio :  0.7518343791181431
